In [1]:
from bs4 import BeautifulSoup
import requests
import re

def get_name(url, id, my_cookies):
    try:
        r = requests.get(url, params = id, cookies = my_cookies)
        r.raise_for_status()
        soup = BeautifulSoup(r.text, 'html.parser')
        
        name_area = soup.find('span', {'class': 'mc-name'})
        if name_area:
            name_string = name_area.string
            name_match = re.match(r'([\u4e00-\u9fa5]+)', name_string)
            return name_match.group(0)
        else:
            # 处理只允许好友查看个人信息的情况
            new_name_area = soup.find('div', {'class': 'mc-hd'}).find('h2')
            name_string = new_name_area.string
            name_match = re.search(r'([\u4e00-\u9fa5]+)', name_string)
            return name_match.group(0)
    except:
        return name_match.group('fail')

def main():
    url = 'http://ccs.seu.edu.cn/ccs/euser.profile.do'
    id = {'uid': ''}
    """
    cookies每次需要重新更换，cookies查找方法：
    在浏览器输入网址：http://ccs.seu.edu.cn/ccs/euser.guest.do?uid=1326062
    其中uid可以更换
    打开”检查“工具，刷新网页，在Network下面的第一条euser.profile.do的Headers部分
    可以看到cookies
    """
    coo = 'JSESSIONID=AEEFCD8D33FBC369A3BE0A015D1EA35D.tomcat2; gr_user_id=01f25979-940d-4a2a-9156-c33aecc65cee; sto-id-20480-sg-ccs.seu.edu.cn=CMIAEBAKHICD; iPlanetDirectoryPro=AQIC5wM2LY4SfczK%2FSlfj%2FHJa3J2jKiF0AnGkwCFuqo5nFc%3D%40AAJTSQACMDI%3D%23; gr_session_id_93fbdae88f63b950=92258400-2850-408d-96d9-52e1c03201b7; gr_session_id_93fbdae88f63b950_92258400-2850-408d-96d9-52e1c03201b7=true'
    my_cookies = {"cookies": coo}
    
    init_id = 1325000
    for i in range(100):
        try:
            
            img_url = 'http://ccs.seu.edu.cn/ccs/attach/printByResource/45/200x200/'
            img_url += str(init_id + i)
            img_url += '.jpg?v=371'
            
            img_src = requests.get(img_url)

            id['uid'] = str(init_id + i)
            person_name = get_name(url, id, my_cookies)
            img_name = r'img/' + str(init_id + i) + person_name + '.jpg'
            with open(img_name, 'wb') as img_file:
                img_file.write(img_src.content)
                
        except:
            print('fail in ' + i)
            continue
    
    print('done')
    
main()

done
